In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp augmentation

# Data augmentation

> Functions to augment the user's dataset with information from official sources.

In [ ]:
#hide
from nbdev.showdoc import *

## Sources of data

`gingado` only lists official data sources by choice. This is meant to provide users with the trust that their dataset will be complemented by reliable sources. Unfortunately, it is not possible at this stage to include *all* official sources - let alone all reliable sources - because that requires substantial manual and maintenance work. `gingado` leverages the existence of the [Statistical Data and Metadata eXchange (SDMX)](https://sdmx.org), an organisation of official data sources that establishes common data and metadata formats, to download data that is relevant (and hopefully also useful) to users.

The function below from the package [simpledmx](https://github.com/dkgaraujo/simpledmx) returns a list of codes corresponding to the data sources available to provide `gingado` users with data through SDMX.

In [ ]:
#export
from simpledmx import *

In [ ]:
list_sdmx_sources()

['ABS',
 'ABS_XML',
 'BBK',
 'BIS',
 'CD2030',
 'ECB',
 'ESTAT',
 'ILO',
 'IMF',
 'INEGI',
 'INSEE',
 'ISTAT',
 'LSD',
 'NB',
 'NBB',
 'OECD',
 'SGR',
 'SPC',
 'STAT_EE',
 'UNICEF',
 'UNSD',
 'WB',
 'WB_WDI']

In [ ]:
#export
import pandas as pd

def augm_with_sdmx(df, start_date, end_date, time_col, freq, sources=None):
    """Downloads relevant data from SDMX sources to complement the original dataset

    Arguments:
      df: a pandas DataFrame
      start_date, end_date: the dates limiting the time period of the desired data from SDMX sources
      time_col: the name of the column in the original dataset that corresponds to time
      freq: the frequency of the desired data from SDMX; for example, 'A' is annual
      sources: the list of SDMX sources or None; a list of possible sources can be obtained by running the function list_sdmx_sources()    
    """
    if start_date is None:
        start_date = df[time_col].min()
    if end_date is None:
        end_date=df[time_col].max()
        
    sdmx_data = get_sdmx_data(
        start_date=start_date,
        end_date=end_date,
        freq=freq,
        sources=sources
        )
    sdmx_data = sdmx_data.dropna(axis=1).sort_index()
    sdmx_data.reset_index(inplace=True)
    sdmx_data['TIME_PERIOD'] = pd.to_datetime(sdmx_data['TIME_PERIOD'])
    if df is None:
        return sdmx_data
    df = df.merge(sdmx_data, how='left', left_on=time_col, right_on='TIME_PERIOD')
    return df

In [ ]:
show_doc(augm_with_sdmx)

<h4 id="augm_with_sdmx" class="doc_header"><code>augm_with_sdmx</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>augm_with_sdmx</code>(**`df`**, **`start_date`**, **`end_date`**, **`time_col`**, **`freq`**, **`sources`**=*`None`*)

Downloads relevant data from SDMX sources to complement the original dataset

Arguments:
  df: a pandas DataFrame
  start_date, end_date: the dates limiting the time period of the desired data from SDMX sources
  time_col: the name of the column in the original dataset that corresponds to time
  freq: the frequency of the desired data from SDMX; for example, 'A' is annual
  sources: the list of SDMX sources or None; a list of possible sources can be obtained by running the function list_sdmx_sources()    

## Using `gingado` to augment a dataset

The data augmentation functionalities of `gingado` is shown below using the [Jordà-Schularick-Taylor Macrohistory Database](https://www.macrohistory.net) on macroeconomics and finance as an example. As a preliminary step, let's import `gingado` and other necessary libraries, and proceed to download the data:

In [ ]:
JST_url = "http://data.macrohistory.net/JST/JSTdatasetR5.dta"
jst = pd.read_stata(JST_url, iterator=False)

jst.tail()

,year,country,iso,ifs,pop,rgdpmad,rgdppc,rconpc,gdp,iy,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
2659,2013.0,USA,USA,111,315820.328999,31571.993947,103.425299,101.892671,16784.851,0.192086,...,0.271035,0.019355,NaN,NaN,NaN,0.023508,0.024601,0.139843,0.212405,-0.065168
2660,2014.0,USA,USA,111,318106.646578,32113.618881,105.186253,104.113597,17527.258,0.196377,...,0.136350,0.019199,NaN,NaN,NaN,0.025408,0.021817,0.130063,0.132729,0.122398
2661,2015.0,USA,USA,111,320413.930388,32800.923063,107.421590,107.192931,18224.780,0.198301,...,-0.000092,0.021124,NaN,NaN,NaN,0.021358,0.021122,0.046193,0.065433,-0.008779
2662,2016.0,USA,USA,111,322705.239927,33078.508719,108.318698,109.333457,18715.040,0.195831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,2017.0,USA,USA,111,324802.861426,33593.446309,110.013284,111.389150,19519.424,0.204547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset is organised in a panel data structure, so the 'year' and 'country' columns are used as indices of the DataFrame. The redundant columns 'iso' and 'ifs' (which are different ways to code the country name) and 'crisisJST_old', which is similar to the target variable 'crisisJST', are dropped.

In [ ]:
jst.drop(labels=['iso', 'ifs', 'crisisJST_old'], axis=1, inplace=True)
jst['year'] = pd.to_datetime(jst['year'], format='%Y')

jst.set_index(['year', 'country'])

,,pop,rgdpmad,rgdppc,rconpc,gdp,iy,cpi,ca,imports,exports,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
year,country,,,,,,,,,,,,,,,,,,,,,
1870-01-01,Australia,1775.000000,3273.239437,13.836157,21.449734,208.780,0.109266,2.708333,-6.147594,36.000,37.000,...,-0.070045,0.071417,NaN,NaN,NaN,0.049118,0.066415,NaN,NaN,NaN
1871-01-01,Australia,1675.000000,3298.507463,13.936864,19.930801,211.560,0.104579,2.666667,5.260774,34.000,46.000,...,0.041654,0.065466,NaN,NaN,NaN,0.048446,0.068193,NaN,NaN,NaN
1872-01-01,Australia,1722.000000,3553.426249,15.044247,21.085006,227.400,0.130438,2.541667,7.867636,38.000,53.000,...,0.108945,0.062997,NaN,NaN,NaN,0.047373,0.069861,NaN,NaN,NaN
1873-01-01,Australia,1769.000000,3823.629169,16.219443,23.254910,266.540,0.124986,2.541667,-11.047833,49.000,50.000,...,0.083086,0.064484,NaN,NaN,NaN,0.046720,0.069842,NaN,NaN,NaN
1874-01-01,Australia,1822.000000,3834.796926,16.268228,23.458050,287.580,0.141960,2.666667,-5.563959,49.000,54.000,...,0.119389,0.063503,NaN,NaN,NaN,0.046533,0.071085,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-01-01,USA,315820.328999,31571.993947,103.425299,101.892671,16784.851,0.192086,173.067206,-426.197000,2768.600,2276.625,...,0.271035,0.019355,NaN,NaN,NaN,0.023508,0.024601,0.139843,0.212405,-0.065168
2014-01-01,USA,318106.646578,32113.618881,105.186253,104.113597,17527.258,0.196377,175.997979,-365.193000,2879.284,2371.027,...,0.136350,0.019199,NaN,NaN,NaN,0.025408,0.021817,0.130063,0.132729,0.122398
2015-01-01,USA,320413.930388,32800.923063,107.421590,107.192931,18224.780,0.198301,176.301162,-407.769000,2786.461,2265.047,...,-0.000092,0.021124,NaN,NaN,NaN,0.021358,0.021122,0.046193,0.065433,-0.008779
